In [ ]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker


diffusion_wrapper = DiffusionWrapper('../text-to-video-ms-1.7b/')
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

In [ ]:
from evaluation_datasets import create_davis_dataset

davis_dataset = create_davis_dataset('../tapvid_davis/tapvid_davis.pkl')

first_video = {}

for data in davis_dataset:
    first_video = data['davis']
    break

video_tensor = torch.tensor(first_video['video'])

In [ ]:
video_features_dict = diffusion_wrapper.extract_video_features(video_tensor, "")

In [ ]:
video_features = diffusion_wrapper.concatenate_video_features({'up_block': video_features_dict['up_block'][0:3]})

In [ ]:
video_features = video_features.permute(0, 2, 3, 1).float()
heatmaps = heatmap_generator.generate(video_features, (93, 137, 0))
tracks = zero_shot_tracker.track(heatmaps)

In [ ]:
zero_shot_tracker.place_marker_in_frames(first_video['video'].squeeze(), tracks)
heatmap_generator.safe_heatmap_as_gif(heatmaps)